<a href="https://colab.research.google.com/github/ten2net/ppo/blob/main/Day27%EF%BC%9A%E4%BA%A4%E6%98%93%E7%AD%96%E7%95%A5%E7%9A%84%E9%83%A8%E7%BD%B2%E3%80%81%E5%AF%A6%E6%99%82%E4%BA%A4%E6%98%93%E8%88%87%E7%9B%A3%E6%8E%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在本課中，我們將學習如何將交易策略部署到實時環境，使用交易API與券商接口進行下單。同時，我們將討論如何實時監控交易策略的性能，以及如何定期評估並調整策略，以確保其有效性和穩定性。[今日 Colab](https://colab.research.google.com/drive/18QmgQgN4fpvuZ92XQnCPvvOnAgwESKXU?usp=sharing)

---

## **一、引言**

![https://ithelp.ithome.com.tw/upload/images/20241012/20120549wrkso35JRU.jpg](https://ithelp.ithome.com.tw/upload/images/20241012/20120549wrkso35JRU.jpg)

### **1.1 交易策略部署的重要性**

- **從理論到實踐**：將已開發和測試的交易策略部署到實時環境，才能真正產生收益。
- **自動化交易**：自動化交易可以減少人工錯誤，提高執行效率。

### **1.2 實時監控與策略評估的必要性**

- **市場變化**：金融市場瞬息萬變，策略需要適應市場環境。
- **風險管理**：實時監控有助於及時發現問題，避免損失擴大。
- **性能優化**：通過評估策略的績效，可以進行調整和優化。

---

## **二、將交易策略部署到實時環境**

### **2.1 部署前的準備工作**

- **策略測試與回測**：確保策略在歷史數據上表現良好。
- **風險評估**：了解策略的最大回撤、風險敞口等。
- **技術準備**：準備好必要的軟件、硬件和網絡環境。

### **2.2 選擇適當的交易平台與API**

- **交易平台**：如 Interactive Brokers、TD Ameritrade、Robinhood 等。
- **API 選擇**：確認平台提供的 API 支持您的編程語言和功能需求。
- **開發者賬戶**：通常需要申請開發者賬戶並獲取 API 密鑰。

### **2.3 環境設置**

- **編程語言**：Python 是常用的語言，支持眾多交易 API。
- **開發環境**：Jupyter Notebook、VS Code、PyCharm 等。
- **依賴庫**：`pandas`、`numpy`、`matplotlib`、`requests`、`websocket-client` 等。

### **2.4 實現自動化交易**

#### **2.4.1 連接到交易 API**

![https://ithelp.ithome.com.tw/upload/images/20241012/201205490g0l4tHB8V.png](https://ithelp.ithome.com.tw/upload/images/20241012/201205490g0l4tHB8V.png)

**示例**：我們這次以 [Alpaca API](https://docs.alpaca.markets/docs/getting-started) 為例（支持美股交易，且有沙盒環境可供測試）

In [ ]:
import alpaca_trade_api as tradeapi

API_KEY = 'YOUR_API_KEY'
API_SECRET = 'YOUR_SECRET_KEY'
BASE_URL = 'https://paper-api.alpaca.markets'

api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

#### **2.4.2 檢查賬戶狀態**


In [ ]:
account = api.get_account()
print(f"可用資金：{account.buying_power}")
print(f"投資組合價值：{account.portfolio_value}")

#### **2.4.3 下單操作**

- **市價單**
就是現在多少價格直接買

In [ ]:
api.submit_order(
    symbol='AAPL',
    qty=10,
    side='buy',
    type='market',
    time_in_force='day'
)

- **限價單**
就是價格達到預定價格時才會進行買賣

In [ ]:
api.submit_order(
    symbol='AAPL',
    qty=10,
    side='sell',
    type='limit',
    limit_price=150.00,
    time_in_force='gtc'  # Good 'Til Canceled
)

#### **2.4.4 查詢訂單狀態**


In [ ]:
orders = api.list_orders(status='all', limit=100)
for order in orders:
    print(f"訂單ID：{order.id}, 狀態：{order.status}, 價格：{order.limit_price}")

#### **2.4.5 實現交易策略**

- **示例策略**：簡單的均線交叉策略

In [ ]:
import pandas as pd

def get_bars(symbol, timeframe, limit):
    barset = api.get_barset(symbol, timeframe, limit=limit)
    bars = barset[symbol]
    data = [{'close': bar.c} for bar in bars]
    df = pd.DataFrame(data)
    return df

def simple_moving_average_strategy(symbol):
    df = get_bars(symbol, 'day', 50)
    df['SMA20'] = df['close'].rolling(window=20).mean()
    df['SMA50'] = df['close'].rolling(window=50).mean()

    if df['SMA20'].iloc[-1] > df['SMA50'].iloc[-1]:
        # 下買單
        api.submit_order(
            symbol=symbol,
            qty=10,
            side='buy',
            type='market',
            time_in_force='day'
        )
        print(f"買入 {symbol}")
    elif df['SMA20'].iloc[-1] < df['SMA50'].iloc[-1]:
        # 下賣單
        api.submit_order(
            symbol=symbol,
            qty=10,
            side='sell',
            type='market',
            time_in_force='day'
        )
        print(f"賣出 {symbol}")
    else:
        print("無交易信號")

## **三、實時監控交易策略的性能**

### **3.1 為什麼需要實時監控**

- **檢測異常**：及時發現策略執行情況是否異常。
- **風險控制**：監控資產淨值變化，防止過度損失。
- **性能分析**：收集實時數據，以便後續分析和優化。

### **3.2 監控的內容**

- **賬戶信息**：可用資金、持倉、盈虧情況。
- **市場數據**：關注的股票價格、成交量等。
- **策略指標**：策略產生的信號、交易次數、勝率等。

### **3.3 實現實時監控**

#### **3.3.1 建立實時數據流**

- **使用 WebSocket**：許多交易 API 提供 WebSocket 接口，可訂閱實時市場數據。

**示例**：訂閱實時行情

In [ ]:
import websocket
import json

def on_open(ws):
    print("連接成功")
    auth_data = {
        "action": "authenticate",
        "data": {
            "key_id": API_KEY,
            "secret_key": API_SECRET
        }
    }
    ws.send(json.dumps(auth_data))
    listen_message = {
        "action": "listen",
        "data": {
            "streams": ["T.AAPL"]
        }
    }
    ws.send(json.dumps(listen_message))

def on_message(ws, message):
    print("接收到消息：", message)

def on_error(ws, error):
    print("發生錯誤：", error)

def on_close(ws):
    print("連接關閉")

socket = "wss://data.alpaca.markets/stream"

ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
ws.run_forever()

#### **3.3.2 監控賬戶與持倉**
​
- **定時檢查**：使用定時任務，每隔一段時間檢查賬戶和持倉狀態。

In [ ]:
import time

while True:
    account = api.get_account()
    positions = api.list_positions()
    print(f"投資組合價值：{account.portfolio_value}")
    for position in positions:
        print(f"持倉：{position.symbol}, 數量：{position.qty}, 盈虧：{position.unrealized_pl}")
    time.sleep(60)  # 每分鐘檢查一次

#### **3.3.3 記錄交易日誌**

- **日誌文件**：將交易行為和策略執行情況記錄到日誌文件中。

In [ ]:
import logging
from datetime import datetime

logging.basicConfig(filename='trading.log', level=logging.INFO)

def log_trade(action, symbol, price, quantity):
    logging.info(f"{datetime.now()} - {action} {quantity} {symbol} at {price}")

#### **3.3.4 可視化監控**

- **實時圖表**：使用 `matplotlib` 或 `plotly` 等庫，繪製實時價格和指標圖表。
- **儀表板**：使用 `Dash` 或 `Streamlit` 等框架，構建網頁儀表板。

**示例**：使用 `matplotlib` 實時更新圖表

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

prices = []

def update(frame):
    current_price = get_current_price('AAPL')
    prices.append(current_price)
    plt.cla()
    plt.plot(prices)
    plt.xlabel('時間')
    plt.ylabel('價格')
    plt.title('AAPL 實時價格')

ani = FuncAnimation(plt.gcf(), update, interval=1000)
plt.show()

### **3.4 建立告警系統**

- **閾值設置**：設定特定的風險閾值，如最大允許損失。
- **自動告警**：當觸發閾值時，通過電子郵件、短信或即時通訊軟件發出告警。

**示例**：當賬戶資產下降超過 5% 時發出告警

In [ ]:
initial_portfolio_value = float(account.portfolio_value)

def check_risk():
    account = api.get_account()
    current_value = float(account.portfolio_value)
    change = (current_value - initial_portfolio_value) / initial_portfolio_value
    if change <= -0.05:
        send_alert(f"資產下降超過 5%，當前價值：{current_value}")

def send_alert(message):
    # 發送電子郵件或其他通知方式
    print("告警：", message)

## **四、定期評估並調整策略**

### **4.1 為什麼需要定期評估**

- **市場變化**：策略可能在不同的市場條件下表現不同。
- **策略老化**：過去有效的策略可能逐漸失效。
- **持續優化**：通過評估找到可以改進的地方。

### **4.2 評估指標**

- **絕對收益**：總收益、年化收益率。
- **風險調整後收益**：夏普比率、索提諾比率。
- **風險指標**：最大回撤、波動率。
- **交易效率**：勝率、盈虧比、交易次數。

### **4.3 策略優化方法**

#### **4.3.1 參數調整**

- **網格搜索**：測試不同參數組合，選擇最佳組合。
- **優化算法**：使用遺傳算法、粒子群算法等進行參數優化。

#### **4.3.2 策略組合**

- **多策略結合**：結合趨勢跟隨、均值回歸等不同策略。
- **資產多樣化**：投資於不同的資產類別，降低風險。

#### **4.3.3 機器學習**

- **監督學習**：使用歷史數據訓練模型，預測價格走勢。
- **強化學習**：讓算法通過與環境交互學習最佳行動策略。

### **4.4 重新回測與驗證**

- **避免過度擬合**：確保模型在未見過的數據上也能表現良好。
- **走勢前移測試（Walk-Forward Testing）**：模擬實際運行，逐步更新模型。
- **交叉驗證**：將數據分為多個部分，輪流作為測試集。

---

## **五、使用 Backtrader 進行實時交易策略部署監控**

那我們也可以將今日的內容與 `Backtrader` 結合，進行實時交易策略部署監控

### **5.1 支持的實時交易平台**

Backtrader 支持與多個實時交易平台集成，包括：

- **Interactive Brokers (IB)**
- **Oanda**
- **Alpaca**

### **5.2 使用 Backtrader 與 Alpaca API 集成**

我們使用 Backtrader 與 Alpaca API 集成，將策略部署到實時環境。以下是具體步驟：

#### **5.2.1 安裝所需的庫**

In [ ]:
pip install backtrader
pip install alpaca-backtrader-api

#### **5.2.2 編寫交易策略**


In [ ]:
import backtrader as bt

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma20 = bt.indicators.SimpleMovingAverage(self.datas[0], period=20)
        self.sma50 = bt.indicators.SimpleMovingAverage(self.datas[0], period=50)

    def next(self):
        if not self.position:
            if self.sma20[0] > self.sma50[0]:
                self.buy(size=10)
                print(f"買入 {self.datas[0]._name}")
        else:
            if self.sma20[0] < self.sma50[0]:
                self.sell(size=10)
                print(f"賣出 {self.datas[0]._name}")

#### **5.2.3 設置實時數據源和交易代理**


In [ ]:
from alpaca_backtrader_api import AlpacaStore

API_KEY = 'YOUR_API_KEY'
API_SECRET = 'YOUR_SECRET_KEY'
USE_POLYGON = False  # 是否使用 Polygon 作為數據源

# 設置 Alpaca Store
store = AlpacaStore(key_id=API_KEY, secret_key=API_SECRET, paper=True, usePolygon=USE_POLYGON)

# 設置 Broker
broker = store.getbroker()

#### **5.2.4 加載實時數據**


In [ ]:
# 設置實時數據源
data = store.getdata(dataname='AAPL', timeframe=bt.TimeFrame.Ticks, compression=1)

# 或者使用分鐘級別的數據
data = store.getdata(dataname='AAPL', timeframe=bt.TimeFrame.Minutes, compression=1)

#### **5.2.5 運行策略**


In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)
cerebro.setbroker(broker)
cerebro.adddata(data)

# 設置初始資金
cerebro.broker.setcash(100000.0)

# 運行策略
cerebro.run()

#### **5.2.6 實時監控**

Backtrader 內置了對實時數據和交易的支持，您可以通過策略中的回調方法（如 `next`、`notify_order`、`notify_trade` 等）監控實時交易的情況。

---

## 六、選擇 Backtrader 還是直接使用 Alpaca API？**

可能有部份讀者開始有點稿混，那我們來分類一下使用情況

### **6.1 使用 Backtrader 的情況**

- **統一的策略開發**：如果您希望在同一個框架中完成策略的開發、回測和實時交易，Backtrader 是一個不錯的選擇。
- **複雜策略**：當您的策略涉及多標的、多時間框架、複雜的指標計算時，Backtrader 的設計更適合管理這些複雜性。
- **可視化分析**：Backtrader 提供了豐富的可視化功能，可以幫助您更好地理解策略的行為。

### **6.2 直接使用 Alpaca API 的情況**

- **簡單策略**：如果您的策略相對簡單，直接使用 Alpaca API 可能更直接，減少學習成本。
- **性能要求**：當您需要更高的性能，或者需要對 API 調用進行精細控制時，直接使用 Alpaca API 更合適。
- **學習目的**：如果您希望深入了解 API 的細節，直接使用 Alpaca API 可以讓您對交易流程有更深的理解。

---

### **6.3 結論**

- **Backtrader** 是一個功能強大的框架，適合開發、回測和部署複雜的交易策略。它與 Alpaca API 集成良好，可以利用 Alpaca 的實時數據和交易功能。
- **Alpaca API** 提供了簡潔易用的接口，適合開發者直接進行實時交易操作，並提供了免費的紙質交易環境。

您可以根據自己的需求和熟悉程度選擇使用 Backtrader 或直接使用 Alpaca API。如果您希望在一個框架內完成策略開發到部署的全流程，並且需要強大的回測和分析功能，那麼 Backtrader 是一個不錯的選擇。如果您更關注交易的執行和性能，並且策略相對簡單，直接使用 Alpaca API 可能更合適。


---

## **七、實踐與注意事項**

### **7.1 實踐步驟**

1. **策略開發與測試**：在歷史數據上開發並測試策略。
2. **部署環境準備**：設置實時交易環境，連接 API。
3. **實時監控**：建立監控系統，確保策略正常運行。
4. **定期評估**：每週或每月評估策略性能，進行必要的調整。

### **7.2 注意事項**

- **API 限制**：了解交易 API 的限制，如速率限制、交易品種等。
- **法規合規**：確保交易行為符合相關法律法規。
- **數據質量**：使用高質量的市場數據，避免錯誤決策。
- **風險管理**：設置止損、止盈，控制單筆交易風險。

---

## **八、總結**

- **策略部署**：將策略從理論帶入實踐，需要技術和風險考量。
- **實時監控**：確保策略按預期運行，及時發現並處理問題。
- **策略評估與調整**：持續改進策略，以適應市場變化。

---

## **九、作業**

1. **策略部署**：選擇一個您開發的交易策略，將其部署到實時環境，進行模擬交易。
2. **實時監控**：建立一個簡單的監控系統，實時監控策略的執行情況和賬戶資產變化。
3. **策略評估**：在運行一週後，評估策略的績效，並提出改進建議。

---

## **十、參考資源**

- **Alpaca API 文檔**: [https://alpaca.markets/docs/api-documentation/](https://alpaca.markets/docs/api-documentation/)
- **WebSocket 客戶端**：[https://pypi.org/project/websocket-client/](https://pypi.org/project/websocket-client/)
- **量化交易書籍**：如《Algorithmic Trading》 by Ernie Chan

---

*通過本課的學習，應該能夠將交易策略部署到實時環境，並且能夠使用交易 API 進行下單。同時，您也學會了如何實時監控策略的性能，並定期評估和調整策略，以適應不斷變化的市場環境。*